# Simple Text Analyzer

ok so here have have some data with some sencetes and tags. We write something and the AI will try to geuss what it's.
 

In [1]:
import json
from io import open
import re
import numpy as np
import pandas as pd

In [2]:
# load data 
with open("data.json", "r") as d:
    data = json.load(d)
data

{'tags': [{'tag': 'greetings',
   'sentences': ['Hello!',
    'Hey',
    "What's up",
    'Nice to meet you',
    'Good morning.',
    'Good evening',
    'Good morning',
    'Good afternoon',
    'Good evening',
    "It's nice to meet you",
    "It's a pleasure to meet you"]},
  {'tag': 'identity',
   'sentences': ['Who are you?',
    'What are you?',
    'What is you name?',
    'What do you do?']},
  {'tag': 'jokes',
   'sentences': ["Did you hear about the mathematician who's afraid of negative numbers?",
    "Why do we tell actors to 'break a leg?'",
    'Helvetica and Times New Roman walk into a bar.',
    'Did you hear about the actor who fell through the floorboards?']}]}

In [3]:
# totalt word list 
wordlist = []
tag_wordlist = dict()
pattern = re.compile('\w+')
for tag in data['tags']:
    tag_wordlist[tag['tag']] = []
    for s in tag['sentences']:
        # print(tag['tag'],[a for a in p.findall(s)])
        wl = [a.lower() for a in pattern.findall(s)]
        tag_wordlist[tag['tag']].extend(wl)
        wordlist.extend(wl)

# wordlist.sort()
wordlist = list(set(wordlist))
wordlist.sort()

TypeError: Index(...) must be called with a collection of some kind, 'k' was passed

In [29]:
# print("tag wl: ",tag_wordlist,'\n')
# print("wl:", wordlist, '\n')
print("len of wordlist:", len(wordlist))
df_tags = pd.DataFrame(enumerate(tag_wordlist))
df_tlk = pd.DataFrame(list(tag_wordlist.keys()), columns=["keys"])
df_tlv = pd.DataFrame(tag_wordlist.values())
df_tlk.join(df_tlv)

len of wordlist: 48


,keys,0,1,2,3,4,5,6,7,8,...,31,32,33,34,35,36,37,38,39,40
0,greetings,hello,hey,what,s,up,nice,to,meet,you,...,you,None,None,None,None,None,None,None,None,None
1,identity,who,are,you,what,are,you,what,is,you,...,None,None,None,None,None,None,None,None,None,None
2,jokes,did,you,hear,about,the,mathematician,who,s,afraid,...,you,hear,about,the,actor,who,fell,through,the,floorboards


So now I'm gonna try to make a matrix of bools/counts,

TagName   | hello | new | why | you    
----------|-------|-----|-----|--------
greetings | 1     | 0   | 1   | 1 or 3 
identity  | 0     | 0   | 0   | 1 or 4 
jokes     | 0     | 1   | 1   | 1 or 2 
 

In [41]:
mat = np.zeros((len(tag_wordlist),len(wordlist)), 'int8')
for iy, word in enumerate(wordlist):
    for ix, tag in enumerate(tag_wordlist):
        for w in tag_wordlist[tag]:
            if w == word:
                mat[ix,iy] += 1


In [43]:
df_mat = pd.DataFrame(mat, columns=wordlist)
df_tlk.join(df_mat)

,keys,a,about,actor,actors,afraid,afternoon,and,are,bar,...,through,times,to,up,walk,we,what,who,why,you
0,greetings,1,0,0,0,0,1,0,0,0,...,0,0,3,1,0,0,1,0,0,3
1,identity,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,3,1,0,4
2,jokes,2,2,1,1,1,0,1,0,1,...,1,1,1,0,1,1,0,2,1,2


OK Now we can start modeling...